# Prepare DWH Schema and Tables

In [337]:
import numpy as np
import pandas as pd
import csv
import locale

In [338]:
# import for date conversion
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

## Define Helper Functions

In [339]:
# convert camel to snake
def convert_to_snake_case(text: str) -> str:
    text = text.strip()
    return ''.join(['_' + s.lower() if (s.isupper() and i > 0 and text[i - 1] != ' ' and not text[i - 1].isupper()) \
                    else '_' if s == ' ' \
                    else s.lower() for i, s in enumerate(text)])

# lower all string columns
def lower_all_object_columns_of_df(df: pd.DataFrame):
    string_columns = df.select_dtypes(include='object').columns
    df[string_columns] = df[string_columns].apply(lambda x: x.str.lower().str.strip())

## Load Data

In [340]:
world_cup_players = pd.read_csv("../../data/input_data/world_cup_players.csv",
                            usecols=['MatchID', 'Team Initials', 'Player Name','Event'])

world_cup_matches = pd.read_csv("../../data/input_data/world_cup_matches.csv",
                            usecols=['MatchID', 'Year', 'Datetime', 'Stadium', 'City', 'Home Team Name','Home Team Goals', 'Away Team Goals', 'Away Team Name',
                                    'Attendance', 'Home Team Initials', 'Away Team Initials'])

world_cup_countries = pd.read_csv("../../data/input_data/fifa_countries_2006.csv",
                            usecols=['Position', 'Team'])

world_cup_players.columns = [convert_to_snake_case(col) for col in world_cup_players.columns]
world_cup_matches.columns = [convert_to_snake_case(col) for col in world_cup_matches.columns]
world_cup_countries.columns = [convert_to_snake_case(col) for col in world_cup_countries.columns]
lower_all_object_columns_of_df(world_cup_players)
lower_all_object_columns_of_df(world_cup_matches)
lower_all_object_columns_of_df(world_cup_countries)

In [341]:
world_cup_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   position  32 non-null     int64 
 1   team      32 non-null     object
dtypes: int64(1), object(1)
memory usage: 644.0+ bytes


In [342]:
world_cup_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   match_id       37784 non-null  int64 
 1   team_initials  37784 non-null  object
 2   player_name    37784 non-null  object
 3   event          9069 non-null   object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [343]:
world_cup_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                852 non-null    int64  
 1   datetime            852 non-null    object 
 2   stadium             852 non-null    object 
 3   city                852 non-null    object 
 4   home_team_name      852 non-null    object 
 5   home_team_goals     852 non-null    int64  
 6   away_team_goals     852 non-null    int64  
 7   away_team_name      852 non-null    object 
 8   attendance          850 non-null    float64
 9   match_id            852 non-null    int64  
 10  home_team_initials  852 non-null    object 
 11  away_team_initials  852 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 80.0+ KB


## Prepare Data and create Tables

In [344]:
# Prepare and create TeamDIM
world_cup_countries = world_cup_countries.rename(columns={'position': 'rank'})
world_cup_countries['team_id'] = range(1, len(world_cup_countries) + 1)

In [345]:
# get missing char players 
players_with_special_char = world_cup_players[world_cup_players['player_name'].str.contains('�')]
players_with_special_char['player_name'].unique()

array(['pel� (edson arantes do nascimento)', 'hugo s�nchez',
       'rom�rio (rom�rio de souza faria)', 'ca�izares', 'm�ller',
       'h�ssler', 'matth�us', 'bj�rnebye', 'z� carlos', 'gon�alves',
       'z� roberto', 'grod�s', 'solskj�r', '�stenstad', 'acu�a',
       'casta�eda', 'wom�', 'etam�', 'pens�e', 'mahouv�', 'olemb�',
       'etxeberr�a', 'ver�n', 'bola�o', 'k�pke', 'w�rns', 'uma�a m.',
       'bola�os c.', 'nu�ez v.', 'c�ceres', 'ca�iza', 'nu�ez', 'caba�as',
       'alvb�ge', 'k�llstr�m', 'allb�ck', 'jo�o ricardo', 'andr� macanga',
       'akw�', 'sim�o', 'z� kalanga', 'loc�', 'lam�', 'fl�vio', 'm�rio',
       'zuberb�hler', 'l�cio', 'kak�', 'luis�o', '�uler', '�eliga',
       'd�ini?', 'matav�', 'milija�', '�igi?', 'to�i?', '�zil',
       'kie�ling', 's�rensen', 'kj�r', 'j�rgensen', 'gr�nkj�r',
       'kr�ldrup', 'f. coentr�o', 'josu�', 'piqu�', 'w�lfli', 'peri�i?',
       'suba�i?', 'j�', 'mand�uki?', 'c. pe�a', 'javi mart�nez',
       'f�bregas', 'ch. ar�nguiz', 'guti�rrez

In [346]:
# clean world_cup_players df and add data to
world_cup_players = world_cup_players.rename(columns={'team_initials': 'player_team_initials'})
world_cup_players['event'] = world_cup_players['event'].fillna('')
world_cup_players['red_cards'] = world_cup_players['event'].str.count('r')
world_cup_players['yellow_cards'] = world_cup_players['event'].str.count(r'(?<!rs)y')
world_cup_players['goals'] = world_cup_players['event'].str.count('g')
# only for 2006 players
world_cup_players['player_name'] = world_cup_players['player_name'].replace(['uma�a m.', 'bola�os c.', 'nu�ez v.', 'c�ceres', 'acu�a', 'ca�iza',
       'nu�ez', 'caba�as', 'alvb�ge', 'k�llstr�m', 'allb�ck',
       'jo�o ricardo', 'andr� macanga', 'akw�', 'sim�o', 'z� kalanga',
       'loc�', 'lam�', 'fl�vio', 'm�rio', 'zuberb�hler', 'l�cio', 'kak�',
       'z� roberto', 'luis�o', 'ca�izares'], ['umaña M.', 'bolaños C.', 'nuñez V.', 'céceres', 'acuña', 'cañiza',
 'nuñez', 'cabañas', 'alvbäge', 'källström', 'allbäck',
 'joão ricardo', 'andrè macanga', 'akwá', 'simão', 'zé kalanga',
 'locó', 'lamá', 'flávio', 'mário', 'zuberbühler', 'lúcio', 'kaká',
 'zé roberto', 'luisão', 'cañizares'])

In [347]:
# get missing char matches 
matches_with_special_char = world_cup_matches[world_cup_matches[['home_team_name', 'away_team_name']].apply(lambda row: row.astype(str).str.contains('�')).any(axis=1)]
pd.Series(matches_with_special_char[['home_team_name', 'away_team_name']].values.ravel()).unique()

array(['argentina', "c�te d'ivoire", 'netherlands',
       'rn">serbia and montenegro', 'portugal', 'brazil', 'korea dpr',
       'japan', 'colombia', 'greece'], dtype=object)

In [348]:
# clean world_cup_matches df and add data to
world_cup_matches['datetime'] = world_cup_matches['datetime'].apply(lambda x: pd.to_datetime(x))
world_cup_matches[['home_team_name', 'away_team_name']] = world_cup_matches[['home_team_name', 'away_team_name']].replace('rn">', '', regex=True)
world_cup_matches = world_cup_matches.replace("c�te d'ivoire", 'ivory coast')
world_cup_matches = world_cup_matches.replace("korea republic", 'south korea')
world_cup_matches = world_cup_matches.replace("ir iran", 'iran')
world_cup_matches[['home_team_name', 'away_team_name']] = world_cup_matches[['home_team_name', 'away_team_name']].replace('usa', 'united states')

In [349]:
merged_matches_players = pd.merge(world_cup_matches, world_cup_players, on='match_id')
merged_matches_players_2006 = merged_matches_players[merged_matches_players['year'] == 2006].copy()
merged_matches_players_2006.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 28218 to 31160
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   year                  2943 non-null   int64         
 1   datetime              2943 non-null   datetime64[ns]
 2   stadium               2943 non-null   object        
 3   city                  2943 non-null   object        
 4   home_team_name        2943 non-null   object        
 5   home_team_goals       2943 non-null   int64         
 6   away_team_goals       2943 non-null   int64         
 7   away_team_name        2943 non-null   object        
 8   attendance            2943 non-null   float64       
 9   match_id              2943 non-null   int64         
 10  home_team_initials    2943 non-null   object        
 11  away_team_initials    2943 non-null   object        
 12  player_team_initials  2943 non-null   object        
 13  player_name       

In [350]:
# Create a mapping between old matchids and new matchids (starting from 1)
unique_ids = merged_matches_players_2006['match_id'].unique()
new_ids = range(1, len(unique_ids) + 1)
id_mapping = dict(zip(unique_ids, new_ids))
merged_matches_players_2006['match_id'] = merged_matches_players_2006['match_id'].map(id_mapping)

In [351]:
# add player match id
merged_matches_players_2006['player_match_id'] = range(1, len(merged_matches_players_2006) + 1)

In [352]:
# add location id to main df
locationdim = merged_matches_players_2006.copy()
locationdim = locationdim.drop_duplicates(subset=['city', 'stadium'])
locationdim['location_id'] = range(1, len(locationdim) + 1)
locationdim = locationdim[['location_id', 'city', 'stadium']] 
merged_matches_players_2006 = pd.merge(merged_matches_players_2006, locationdim, on=['city', 'stadium'])

In [353]:
# add player id to main df
playerdim = merged_matches_players_2006.copy()
# all players in our dataset have different names, Júnior changed team, Asamoah changed shirt number
playerdim = playerdim.drop_duplicates(subset=['player_name'])
playerdim['player_id'] = range(1, len(playerdim) + 1)
playerdim = playerdim[['player_id', 'player_name']]
merged_matches_players_2006 = pd.merge(merged_matches_players_2006, playerdim, on=['player_name'])

In [354]:
# get max time id for our time ids
import pyexasol
C = pyexasol.connect_local_config('standard_exasol', config_path='../../.pyexasol.ini', protocol_version = 1)
max_time_id = int(C.export_to_list(
    """SELECT MAX(ID) as max_id
        FROM AOL_SCHEMA.TIMEDIM"""
)[0][0])
C.close()
timedim = merged_matches_players_2006.copy()
timedim = timedim.drop_duplicates(subset=['datetime'])
timedim['time_id'] = range(max_time_id + 1, max_time_id + len(timedim) + 1)
timedim = timedim[['time_id', 'datetime']]
merged_matches_players_2006 = pd.merge(merged_matches_players_2006, timedim, on=['datetime'])

In [355]:
home_matches = merged_matches_players_2006.merge(world_cup_countries, left_on='home_team_name', right_on='team', how='inner')
home_matches = home_matches[home_matches['home_team_initials'] == home_matches['player_team_initials']]
away_matches = merged_matches_players_2006.merge(world_cup_countries, left_on='away_team_name', right_on='team', how='inner')
away_matches = away_matches[away_matches['away_team_initials'] == away_matches['player_team_initials']]

# create final dataframe for fact table
merged_df = pd.concat([home_matches, away_matches])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   year                  2943 non-null   int64         
 1   datetime              2943 non-null   datetime64[ns]
 2   stadium               2943 non-null   object        
 3   city                  2943 non-null   object        
 4   home_team_name        2943 non-null   object        
 5   home_team_goals       2943 non-null   int64         
 6   away_team_goals       2943 non-null   int64         
 7   away_team_name        2943 non-null   object        
 8   attendance            2943 non-null   float64       
 9   match_id              2943 non-null   int64         
 10  home_team_initials    2943 non-null   object        
 11  away_team_initials    2943 non-null   object        
 12  player_team_initials  2943 non-null   object        
 13  player_name           2

In [356]:
# Prepare and create teamdim
teamdim = merged_df.copy()
teamdim = teamdim.drop_duplicates(subset='team_id')
teamdim = teamdim.rename(columns={'team_id': 'id'})
teamdim = teamdim[['id', 'team', 'rank']]
teamdim.to_csv('../../data/query_data/teamdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
teamdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 0 to 2897
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      32 non-null     int64 
 1   team    32 non-null     object
 2   rank    32 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ KB


In [357]:
# Prepare and create player_match_statisticsdim
player_match_statisticsdim = merged_df.copy()
player_match_statisticsdim = player_match_statisticsdim.rename(columns={'player_match_id': 'id'})
player_match_statisticsdim = player_match_statisticsdim[['id', 'goals', 'red_cards', 'yellow_cards']]
player_match_statisticsdim.to_csv('../../data/query_data/player_matchdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
player_match_statisticsdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            2943 non-null   int64
 1   goals         2943 non-null   int64
 2   red_cards     2943 non-null   int64
 3   yellow_cards  2943 non-null   int64
dtypes: int64(4)
memory usage: 115.0 KB


In [358]:
# Prepare and create matchdim
matchdim = merged_df.copy()
matchdim = matchdim.drop_duplicates(subset='match_id')
matchdim = matchdim.rename(columns={'match_id': 'id'})
matchdim = matchdim[['id', 'home_team_name', 'away_team_name', 'home_team_goals', 'away_team_goals', 'attendance']]
matchdim.to_csv('../../data/query_data/matchdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
matchdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 0 to 2897
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               64 non-null     int64  
 1   home_team_name   64 non-null     object 
 2   away_team_name   64 non-null     object 
 3   home_team_goals  64 non-null     int64  
 4   away_team_goals  64 non-null     int64  
 5   attendance       64 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 3.5+ KB


In [359]:
# Prepare and create locationdim
locationdim = merged_df.copy()
locationdim = locationdim.drop_duplicates(subset=['location_id'])
locationdim = locationdim.rename(columns={'location_id': 'id'})
locationdim = locationdim[['id', 'city', 'stadium']]
locationdim.to_csv('../../data/query_data/locationdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
locationdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0 to 1356
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       12 non-null     int64 
 1   city     12 non-null     object
 2   stadium  12 non-null     object
dtypes: int64(1), object(2)
memory usage: 384.0+ bytes


In [360]:
# Prepare and create playerdim
playerdim = merged_df.copy()
playerdim = playerdim.drop_duplicates(subset='player_id')
playerdim = playerdim.rename(columns={'player_id': 'id'})
playerdim = playerdim[['id', 'player_name']]
playerdim.to_csv('../../data/query_data/playerdim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
playerdim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 735 entries, 0 to 2432
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           735 non-null    int64 
 1   player_name  735 non-null    object
dtypes: int64(1), object(1)
memory usage: 17.2+ KB


In [361]:
# Prepare and create new timedim data
timedim = merged_df.copy()
timedim = timedim.drop_duplicates(subset='time_id')
timedim = timedim.rename(columns={'time_id': 'id'})
timedim = timedim[['id', 'datetime']]
timedim['year'] = timedim['datetime'].dt.year
timedim['month'] = timedim['datetime'].dt.strftime('%B')
timedim['calendar week'] = timedim['datetime'].dt.isocalendar().week
timedim['day of the week'] = timedim['datetime'].dt.dayofweek
timedim['weekday'] = timedim['datetime'].dt.strftime('%A')
timedim['day of the month'] = timedim['datetime'].dt.day
timedim['day of the year'] = timedim['datetime'].dt.dayofyear
timedim['hour'] = timedim['datetime'].dt.hour
timedim['minute'] = timedim['datetime'].dt.minute
timedim['second'] = timedim['datetime'].dt.second
timedim = timedim.drop(columns=['datetime'])
lower_all_object_columns_of_df(timedim)
timedim.to_csv('../../data/query_data/timedim.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
timedim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, 0 to 2897
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                56 non-null     int64 
 1   year              56 non-null     int32 
 2   month             56 non-null     object
 3   calendar week     56 non-null     UInt32
 4   day of the week   56 non-null     int32 
 5   weekday           56 non-null     object
 6   day of the month  56 non-null     int32 
 7   day of the year   56 non-null     int32 
 8   hour              56 non-null     int32 
 9   minute            56 non-null     int32 
 10  second            56 non-null     int32 
dtypes: UInt32(1), int32(7), int64(1), object(2)
memory usage: 3.6+ KB


In [362]:
# Prepare and create fact table
match_facts = merged_df.copy()
match_facts = match_facts[['match_id', 'player_match_id', 'player_id', 'time_id', 'team_id', 'location_id']]
match_facts.to_csv('../../data/query_data/match_facts.csv', quoting=csv.QUOTE_ALL,
                  index=False, header=False, lineterminator='\n')
match_facts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   match_id         2943 non-null   int64
 1   player_match_id  2943 non-null   int64
 2   player_id        2943 non-null   int64
 3   time_id          2943 non-null   int64
 4   team_id          2943 non-null   int64
 5   location_id      2943 non-null   int64
dtypes: int64(6)
memory usage: 160.9 KB
